In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import flask
from flask import jsonify
import csv
import numpy
import datetime


#Function that calculates win % or returns 50% if no record
def split_record(record_string):
    cleaned=record_string.strip(',').split('-')
    if int(cleaned[0])+int(cleaned[1])==0:
        return 0
    else:
        return round(int(cleaned[0])/(int(cleaned[1])+int(cleaned[0])),2)


def preview_ext(url,home_abv,year,month,day,gno):

    #Scrape Pandas Tables
    tables=pd.read_html(url)

    #Names
    home_name=tables[1][1][0].split('2019 ')[1]
    away_name=tables[0][1][0].split('2019 ')[1]


    #Number of Games Played This Season/Test if We Should Record Data
    game_no=int(tables[0][1][3])

    #Scrape Soup
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')


    #Ugly Away Pitcher Data
    away_pitcher=str(soup.find_all('h2')[2]).split()

    #Away Pitcher Handedness
    away_pitcher_hand=away_pitcher[5]
    if away_pitcher_hand=='RHP,':
        away_pitcher_rh=1
    else: 
        away_pitcher_rh=0


    #Away Pitcher Record
    try:
        away_pitcher_record=float(split_record(away_pitcher[6]))
    except: 
        away_pitcher_record=0

    #Away Pitcher ERA
    try:
        away_pitcher_era=float(away_pitcher[7].strip(')</h2>'))
    except:
        away_pitcher_era=0

    #Away Pitcher Innings Pitched, may contain prior year data if before May
    try:
        away_pitcher_ip=float(tables[8]['IP'][0])
    except:
        away_pitcher_ip=0

    #Ugly Home Pitcher Data
    home_pitcher=str(soup.find_all('h2')[5]).split()

    #Home Pitcher Handedness
    home_pitcher_hand=home_pitcher[5]
    if home_pitcher_hand=='RHP,':
        home_pitcher_rh=1
    elif home_pitcher_hand=='LHP,':
        home_pitcher_rh=0
    else:
        home_pitcher_rh=2

    #Home Pitcher Record
    try:
        home_pitcher_record=float(split_record(home_pitcher[6]))
    except:
        home_pitcher_record=0

    #Home Pitcher ERA
    try:
        home_pitcher_era=float(home_pitcher[7].strip(')</h2>'))
    except:
        home_pitcher_era=0

    #Home Pitcher Innings Pitched, may contain prior year data if before May
    try:
        home_pitcher_ip=float(tables[5]['IP'][0])
    except:
        home_pitcher_ip=0


    #Away Team Main Table
    team=tables[0]

    #Away Team Overall Win/Loss Record
    away_record=split_record(team[1][1])


    #Away Team Win/Loss Record in Last 10 Games
    away_last_ten=split_record(team[1][10])

    #Away Team Record as Away Team 
    away_venue_record=split_record(team[1][14])

    #Away Team Record vs Home Team Pitcher Type
    if home_pitcher_rh==1:
        away_pitcher_type_record=split_record(team[1][16])
    else:
        split_record(team[1][17])

    #Home Team Main Table
    team=tables[1]

    #Home Team Overall Win/Loss Record
    home_record=split_record(team[1][1])


    #Home Team Win/Loss Record in Last 10 Games
    home_last_ten=split_record(team[1][10])

    #Home Team Record as Home Team 
    home_venue_record=split_record(team[1][13])

    #Home Team Record vs Away Team Pitcher Type
    if away_pitcher_rh==1:
        home_pitcher_type_record=split_record(team[1][16])
    else:
        home_pitcher_type_record=split_record(team[1][17])

    #Away Team Batting Table, may contain prior year data if before May
    batting=tables[4]

    #Away Team Ops vs Home Pitcher Type
    if home_pitcher_rh==1:
        away_ops_vs_pitcher_type=batting['OPS vRH'][len(batting)-1]
    else:
        away_ops_vs_pitcher_type=batting['OPS vLH'][len(batting)-1]

    #Home Team Batting Table, may contain prior year data if before May
    batting=tables[7]

    #Home Team Ops vs Away Pitcher Type
    if away_pitcher_rh==1:
        home_ops_vs_pitcher_type=batting['OPS vRH'][len(batting)-1]
    else:
        home_ops_vs_pitcher_type=batting['OPS vLH'][len(batting)-1]

    #Home Team Away Team Matchup Record
    head2head=tables[2]

    #Number of Matchups this Season
    try:
        matchup_count=head2head.loc[head2head['Winner'] == 'Away'].index[0]

        #Home Team Win % vs this Away Team
        season_matchups=head2head.head(matchup_count)
        home_matchup_wins=len(season_matchups.loc[head2head['Winner']==home_abv])
        home_matchup_record=home_matchup_wins/matchup_count
    except:
        matchup_count=0
        home_matchup_record=.5

    preview_data= [home_name,away_name,game_no,away_pitcher_record,away_pitcher_era,away_pitcher_ip,
    home_pitcher_record,home_pitcher_era,home_pitcher_ip,
    away_record,away_last_ten,away_venue_record,away_pitcher_type_record,
    home_record,home_last_ten,home_venue_record,home_pitcher_type_record,
    away_ops_vs_pitcher_type,home_ops_vs_pitcher_type,
    matchup_count,home_matchup_record]


    return preview_data 

columns = ["home_name","away_name","game_no","away_pitcher_record","away_pitcher_era","away_pitcher_ip",
"home_pitcher_record","home_pitcher_era","home_pitcher_ip",
"away_record","away_last_ten","away_venue_record","away_pitcher_type_record",
"home_record","home_last_ten","home_venue_record","home_pitcher_type_record",
"away_ops_vs_pitcher_type","home_ops_vs_pitcher_type",
"matchup_count","home_matchup_record"]  


teams=['ATL','ARI','BAL','BOS','CHN','CHA','CIN','CLE','COL','DET','HOU','KCA','ANA','LAN','MIA','MIL','MIN','NYN','NYA','OAK','PHI','PIT','SDN','SFN','SEA','SLN','TBA','TEX','TOR','WAS']

# getting current date and time
date_list = datetime.date.today()
date_list=list(date_list.timetuple())
date_list


[2019, 7, 8, 0, 0, 0, 0, 189, -1]

In [6]:
df=[]
for t in teams:
    try:
        home_abv=t
        day= '02'
        month='07'
        year='2019'
#         year= f'{date_list[0]}'
#         month= f'{date_list[1]}'
#         day= f'{date_list[2]}'
        gno='0'
        url=f'https://www.baseball-reference.com/previews/{year}/{home_abv}{year}{month}{day}{gno}.shtml'
        df.append(preview_ext(url,home_abv,year,month,day,gno))

     
    except:
        pass
df

[['Chicago White Sox',
  'Detroit Tigers',
  81,
  0.45,
  3.7,
  101.2,
  0.36,
  6.1,
  92.2,
  0.34,
  0.1,
  0.39,
  0.32,
  0.48,
  0.4,
  0.55,
  0.54,
  0.659,
  0.735,
  5,
  0.0],
 ['Cincinnati Reds',
  'Milwaukee Brewers',
  86,
  0.67,
  4.4,
  57.0,
  0.45,
  3.36,
  85.2,
  0.54,
  0.6,
  0.47,
  0.58,
  0.46,
  0.4,
  0.53,
  0.45,
  0.778,
  0.7140000000000001,
  10,
  0.3],
 ['Colorado Rockies',
  'Houston Astros',
  86,
  0,
  0,
  109.2,
  0.73,
  4.29,
  115.1,
  0.62,
  0.5,
  0.54,
  0.57,
  0.52,
  0.4,
  0.59,
  0.53,
  0.7879999999999999,
  0.8109999999999999,
  0,
  0.5],
 ['Kansas City Royals',
  'Cleveland Indians',
  84,
  0.5,
  3.55,
  119.0,
  0.36,
  5.23,
  94.2,
  0.54,
  0.6,
  0.5,
  0.51,
  0.34,
  0.3,
  0.39,
  0.34,
  0.732,
  0.728,
  6,
  0.0],
 ['Los Angeles Dodgers',
  'Arizona Diamondbacks',
  87,
  0.4,
  6.1,
  38.1,
  0.6,
  3.08,
  52.2,
  0.5,
  0.5,
  0.55,
  0.48,
  0.66,
  0.6,
  0.79,
  0.66,
  0.738,
  0.828,
  10,
  0.0],
 ['New Y

In [7]:
d1 = {'games': df}
df2 = pd.DataFrame(d1)
print (df2)

                                                games
0   [Chicago White Sox, Detroit Tigers, 81, 0.45, ...
1   [Cincinnati Reds, Milwaukee Brewers, 86, 0.67,...
2   [Colorado Rockies, Houston Astros, 86, 0, 0, 1...
3   [Kansas City Royals, Cleveland Indians, 84, 0....
4   [Los Angeles Dodgers, Arizona Diamondbacks, 87...
5   [New York Mets, New York Yankees, 83, 0.62, 4....
6   [Oakland Athletics, Minnesota Twins, 84, 0.77,...
7   [Pittsburgh Pirates, Chicago Cubs, 86, 0.58, 3...
8   [Seattle Mariners, St. Louis Cardinals, 83, 0....
9   [Tampa Bay Rays, Baltimore Orioles, 85, 0.23, ...
10  [Toronto Blue Jays, Boston Red Sox, 85, 0.71, ...


In [8]:
df2[["home_name","away_name","game_no","away_pitcher_record","away_pitcher_era","away_pitcher_ip","home_pitcher_record","home_pitcher_era","home_pitcher_ip",
"away_record","away_last_ten","away_venue_record","away_pitcher_type_record",
"home_record","home_last_ten","home_venue_record","home_pitcher_type_record",
"away_ops_vs_pitcher_type","home_ops_vs_pitcher_type",
"matchup_count","home_matchup_record"]] = pd.DataFrame(df2.games.values.tolist(), index= df2.index)
print (df2)

                                                games            home_name  \
0   [Chicago White Sox, Detroit Tigers, 81, 0.45, ...    Chicago White Sox   
1   [Cincinnati Reds, Milwaukee Brewers, 86, 0.67,...      Cincinnati Reds   
2   [Colorado Rockies, Houston Astros, 86, 0, 0, 1...     Colorado Rockies   
3   [Kansas City Royals, Cleveland Indians, 84, 0....   Kansas City Royals   
4   [Los Angeles Dodgers, Arizona Diamondbacks, 87...  Los Angeles Dodgers   
5   [New York Mets, New York Yankees, 83, 0.62, 4....        New York Mets   
6   [Oakland Athletics, Minnesota Twins, 84, 0.77,...    Oakland Athletics   
7   [Pittsburgh Pirates, Chicago Cubs, 86, 0.58, 3...   Pittsburgh Pirates   
8   [Seattle Mariners, St. Louis Cardinals, 83, 0....     Seattle Mariners   
9   [Tampa Bay Rays, Baltimore Orioles, 85, 0.23, ...       Tampa Bay Rays   
10  [Toronto Blue Jays, Boston Red Sox, 85, 0.71, ...    Toronto Blue Jays   

               away_name  game_no  away_pitcher_record  away_pi

In [9]:
df3 = pd.DataFrame(df2['games'].values.tolist(), columns=["home_name","away_name","game_no","away_pitcher_record","away_pitcher_era","away_pitcher_ip",
"home_pitcher_record","home_pitcher_era","home_pitcher_ip",
"away_record","away_last_ten","away_venue_record","away_pitcher_type_record",
"home_record","home_last_ten","home_venue_record","home_pitcher_type_record",
"away_ops_vs_pitcher_type","home_ops_vs_pitcher_type",
"matchup_count","home_matchup_record"])
df3

,home_name,away_name,game_no,away_pitcher_record,away_pitcher_era,away_pitcher_ip,home_pitcher_record,home_pitcher_era,home_pitcher_ip,away_record,...,away_venue_record,away_pitcher_type_record,home_record,home_last_ten,home_venue_record,home_pitcher_type_record,away_ops_vs_pitcher_type,home_ops_vs_pitcher_type,matchup_count,home_matchup_record
0,Chicago White Sox,Detroit Tigers,81,0.45,3.70,101.2,0.36,6.10,92.2,0.34,...,0.39,0.32,0.48,0.4,0.55,0.54,0.659,0.735,5,0.000000
1,Cincinnati Reds,Milwaukee Brewers,86,0.67,4.40,57.0,0.45,3.36,85.2,0.54,...,0.47,0.58,0.46,0.4,0.53,0.45,0.778,0.714,10,0.300000
2,Colorado Rockies,Houston Astros,86,0.00,0.00,109.2,0.73,4.29,115.1,0.62,...,0.54,0.57,0.52,0.4,0.59,0.53,0.788,0.811,0,0.500000
3,Kansas City Royals,Cleveland Indians,84,0.50,3.55,119.0,0.36,5.23,94.2,0.54,...,0.50,0.51,0.34,0.3,0.39,0.34,0.732,0.728,6,0.000000
4,Los Angeles Dodgers,Arizona Diamondbacks,87,0.40,6.10,38.1,0.60,3.08,52.2,0.50,...,0.55,0.48,0.66,0.6,0.79,0.66,0.738,0.828,10,0.000000
5,New York Mets,New York Yankees,83,0.62,4.34,64.1,0.55,4.51,107.2,0.66,...,0.62,0.69,0.45,0.3,0.57,0.45,0.838,0.826,2,0.000000
6,Oakland Athletics,Minnesota Twins,84,0.77,0.73,85.2,0.67,4.03,29.0,0.64,...,0.63,0.65,0.54,0.7,0.56,0.48,0.818,0.733,0,0.500000
7,Pittsburgh Pirates,Chicago Cubs,86,0.58,3.36,88.1,0.46,4.27,90.2,0.53,...,0.40,0.56,0.48,0.7,0.49,0.49,0.793,0.782,4,0.500000
8,Seattle Mariners,St. Louis Cardinals,83,0.44,4.75,85.1,0.00,0.00,3.0,0.50,...,0.42,0.51,0.42,0.5,0.40,0.46,0.713,0.772,0,0.500000
9,Tampa Bay Rays,Baltimore Orioles,85,0.23,4.91,84.1,0.80,0.43,100.0,0.29,...,0.31,0.29,0.58,0.6,0.55,0.57,0.701,0.763,6,0.000000


In [10]:
X = df3.drop(["home_name","away_name"], axis=1)
X

,game_no,away_pitcher_record,away_pitcher_era,away_pitcher_ip,home_pitcher_record,home_pitcher_era,home_pitcher_ip,away_record,away_last_ten,away_venue_record,away_pitcher_type_record,home_record,home_last_ten,home_venue_record,home_pitcher_type_record,away_ops_vs_pitcher_type,home_ops_vs_pitcher_type,matchup_count,home_matchup_record
0,81,0.45,3.70,101.2,0.36,6.10,92.2,0.34,0.1,0.39,0.32,0.48,0.4,0.55,0.54,0.659,0.735,5,0.000000
1,86,0.67,4.40,57.0,0.45,3.36,85.2,0.54,0.6,0.47,0.58,0.46,0.4,0.53,0.45,0.778,0.714,10,0.300000
2,86,0.00,0.00,109.2,0.73,4.29,115.1,0.62,0.5,0.54,0.57,0.52,0.4,0.59,0.53,0.788,0.811,0,0.500000
3,84,0.50,3.55,119.0,0.36,5.23,94.2,0.54,0.6,0.50,0.51,0.34,0.3,0.39,0.34,0.732,0.728,6,0.000000
4,87,0.40,6.10,38.1,0.60,3.08,52.2,0.50,0.5,0.55,0.48,0.66,0.6,0.79,0.66,0.738,0.828,10,0.000000
5,83,0.62,4.34,64.1,0.55,4.51,107.2,0.66,0.9,0.62,0.69,0.45,0.3,0.57,0.45,0.838,0.826,2,0.000000
6,84,0.77,0.73,85.2,0.67,4.03,29.0,0.64,0.5,0.63,0.65,0.54,0.7,0.56,0.48,0.818,0.733,0,0.500000
7,86,0.58,3.36,88.1,0.46,4.27,90.2,0.53,0.4,0.40,0.56,0.48,0.7,0.49,0.49,0.793,0.782,4,0.500000
8,83,0.44,4.75,85.1,0.00,0.00,3.0,0.50,0.4,0.42,0.51,0.42,0.5,0.40,0.46,0.713,0.772,0,0.500000
9,85,0.23,4.91,84.1,0.80,0.43,100.0,0.29,0.3,0.31,0.29,0.58,0.6,0.55,0.57,0.701,0.763,6,0.000000


In [11]:
# Scale the data using MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X)


In [12]:
X_scaled = X_scaler.transform(X)


In [13]:
import pickle

# Load from file
with open('finalized_model.sav', 'rb') as file:  
    pickle_model = pickle.load(file)

# Predict target values
Ypredict = pickle_model.predict(X_scaled)  
print(Ypredict)

['1' '1' '0' '0' '1' '0' '0' '0' '0' '1' '0']


/Users/Monchis/anaconda3/envs/PythonData/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.1 when using version 0.21.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [14]:
predict_outcome = Ypredict
predict_outcome

array(['1', '1', '0', '0', '1', '0', '0', '0', '0', '1', '0'],
      dtype=object)

In [15]:
df3['predicted_outcome']= predict_outcome
df3 = df3.astype({"predicted_outcome": int})
df3

,home_name,away_name,game_no,away_pitcher_record,away_pitcher_era,away_pitcher_ip,home_pitcher_record,home_pitcher_era,home_pitcher_ip,away_record,...,away_pitcher_type_record,home_record,home_last_ten,home_venue_record,home_pitcher_type_record,away_ops_vs_pitcher_type,home_ops_vs_pitcher_type,matchup_count,home_matchup_record,predicted_outcome
0,Chicago White Sox,Detroit Tigers,81,0.45,3.70,101.2,0.36,6.10,92.2,0.34,...,0.32,0.48,0.4,0.55,0.54,0.659,0.735,5,0.000000,1
1,Cincinnati Reds,Milwaukee Brewers,86,0.67,4.40,57.0,0.45,3.36,85.2,0.54,...,0.58,0.46,0.4,0.53,0.45,0.778,0.714,10,0.300000,1
2,Colorado Rockies,Houston Astros,86,0.00,0.00,109.2,0.73,4.29,115.1,0.62,...,0.57,0.52,0.4,0.59,0.53,0.788,0.811,0,0.500000,0
3,Kansas City Royals,Cleveland Indians,84,0.50,3.55,119.0,0.36,5.23,94.2,0.54,...,0.51,0.34,0.3,0.39,0.34,0.732,0.728,6,0.000000,0
4,Los Angeles Dodgers,Arizona Diamondbacks,87,0.40,6.10,38.1,0.60,3.08,52.2,0.50,...,0.48,0.66,0.6,0.79,0.66,0.738,0.828,10,0.000000,1
5,New York Mets,New York Yankees,83,0.62,4.34,64.1,0.55,4.51,107.2,0.66,...,0.69,0.45,0.3,0.57,0.45,0.838,0.826,2,0.000000,0
6,Oakland Athletics,Minnesota Twins,84,0.77,0.73,85.2,0.67,4.03,29.0,0.64,...,0.65,0.54,0.7,0.56,0.48,0.818,0.733,0,0.500000,0
7,Pittsburgh Pirates,Chicago Cubs,86,0.58,3.36,88.1,0.46,4.27,90.2,0.53,...,0.56,0.48,0.7,0.49,0.49,0.793,0.782,4,0.500000,0
8,Seattle Mariners,St. Louis Cardinals,83,0.44,4.75,85.1,0.00,0.00,3.0,0.50,...,0.51,0.42,0.5,0.40,0.46,0.713,0.772,0,0.500000,0
9,Tampa Bay Rays,Baltimore Orioles,85,0.23,4.91,84.1,0.80,0.43,100.0,0.29,...,0.29,0.58,0.6,0.55,0.57,0.701,0.763,6,0.000000,1


In [16]:
df3["predicted_winner"] = ["Home Team" if ele  == 1 else "Away Team" for ele in df3["predicted_outcome"]]
df3


,home_name,away_name,game_no,away_pitcher_record,away_pitcher_era,away_pitcher_ip,home_pitcher_record,home_pitcher_era,home_pitcher_ip,away_record,...,home_record,home_last_ten,home_venue_record,home_pitcher_type_record,away_ops_vs_pitcher_type,home_ops_vs_pitcher_type,matchup_count,home_matchup_record,predicted_outcome,predicted_winner
0,Chicago White Sox,Detroit Tigers,81,0.45,3.70,101.2,0.36,6.10,92.2,0.34,...,0.48,0.4,0.55,0.54,0.659,0.735,5,0.000000,1,Home Team
1,Cincinnati Reds,Milwaukee Brewers,86,0.67,4.40,57.0,0.45,3.36,85.2,0.54,...,0.46,0.4,0.53,0.45,0.778,0.714,10,0.300000,1,Home Team
2,Colorado Rockies,Houston Astros,86,0.00,0.00,109.2,0.73,4.29,115.1,0.62,...,0.52,0.4,0.59,0.53,0.788,0.811,0,0.500000,0,Away Team
3,Kansas City Royals,Cleveland Indians,84,0.50,3.55,119.0,0.36,5.23,94.2,0.54,...,0.34,0.3,0.39,0.34,0.732,0.728,6,0.000000,0,Away Team
4,Los Angeles Dodgers,Arizona Diamondbacks,87,0.40,6.10,38.1,0.60,3.08,52.2,0.50,...,0.66,0.6,0.79,0.66,0.738,0.828,10,0.000000,1,Home Team
5,New York Mets,New York Yankees,83,0.62,4.34,64.1,0.55,4.51,107.2,0.66,...,0.45,0.3,0.57,0.45,0.838,0.826,2,0.000000,0,Away Team
6,Oakland Athletics,Minnesota Twins,84,0.77,0.73,85.2,0.67,4.03,29.0,0.64,...,0.54,0.7,0.56,0.48,0.818,0.733,0,0.500000,0,Away Team
7,Pittsburgh Pirates,Chicago Cubs,86,0.58,3.36,88.1,0.46,4.27,90.2,0.53,...,0.48,0.7,0.49,0.49,0.793,0.782,4,0.500000,0,Away Team
8,Seattle Mariners,St. Louis Cardinals,83,0.44,4.75,85.1,0.00,0.00,3.0,0.50,...,0.42,0.5,0.40,0.46,0.713,0.772,0,0.500000,0,Away Team
9,Tampa Bay Rays,Baltimore Orioles,85,0.23,4.91,84.1,0.80,0.43,100.0,0.29,...,0.58,0.6,0.55,0.57,0.701,0.763,6,0.000000,1,Home Team


In [17]:
display_data = df3[["home_name","away_name", "predicted_winner"]].copy()

In [18]:
dict = display_data.to_dict(orient='records')
dict

[{'home_name': 'Chicago White Sox',
  'away_name': 'Detroit Tigers',
  'predicted_winner': 'Home Team'},
 {'home_name': 'Cincinnati Reds',
  'away_name': 'Milwaukee Brewers',
  'predicted_winner': 'Home Team'},
 {'home_name': 'Colorado Rockies',
  'away_name': 'Houston Astros',
  'predicted_winner': 'Away Team'},
 {'home_name': 'Kansas City Royals',
  'away_name': 'Cleveland Indians',
  'predicted_winner': 'Away Team'},
 {'home_name': 'Los Angeles Dodgers',
  'away_name': 'Arizona Diamondbacks',
  'predicted_winner': 'Home Team'},
 {'home_name': 'New York Mets',
  'away_name': 'New York Yankees',
  'predicted_winner': 'Away Team'},
 {'home_name': 'Oakland Athletics',
  'away_name': 'Minnesota Twins',
  'predicted_winner': 'Away Team'},
 {'home_name': 'Pittsburgh Pirates',
  'away_name': 'Chicago Cubs',
  'predicted_winner': 'Away Team'},
 {'home_name': 'Seattle Mariners',
  'away_name': 'St. Louis Cardinals',
  'predicted_winner': 'Away Team'},
 {'home_name': 'Tampa Bay Rays',
  'away_

In [19]:
display_data.to_csv ("static/js/display_data.csv")